In [20]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 30)

import xlrd, xlwt

In [21]:
stops_data = pd.read_excel("Остановки конвертера (нагнетателя) январь-июль.xlsx")

In [22]:
stops_data

,Название Цеха,Т.Группа.План,Статус КТГ,Текст Соотв.ТМ,Техместо,Название,Сообщение,ТекстГруппКодов,Текст кода код.,Ед. оборудов.,Описание,Продолж.простоя мин,ПродолжПростоя,Единица,НачалоНеисправн,...,ВремяУстрНеиспр,Отказ,Код ABC,Вид сообщения,Создал,Время сообщения,Дата сообщения,СистСтатусЗаказа,Заказ,ПользовСтатус,СистемнСтатус,СоотвТехМесто,Ответств.организация,Ответств.служба,Ответств.подраздел
0,СП,СП:ЦВКС,КТГ,Комплекс конвертера N 1,CH-SPP-KKC/OVS-006,Комплекс конвертера N 1,17140326,Технологические неплановые,Прочее,NaN,додувка на С+Р,0.90,0.01,Ч,2020-06-30,...,06:37:18,X,NaN,M1,PMN_INT_USER,06:38:44,2020-06-30,NaN,NaN,НРЗБ БЛОК,СООТ,CH-SPP-KKC/OVS-006,"ПАО ""СЕВЕРСТАЛЬ""",NaN,NaN
1,СП,СП:ЦВКС,КТГ,Комплекс конвертера N 1,CH-SPP-KKC/OVS-006,Комплекс конвертера N 1,17138411,Технологические плановые,Межплавочный период,NaN,межплавочный период,24.12,0.40,Ч,2020-06-29,...,23:22:32,X,NaN,M1,PMN_INT_USER,23:24:48,2020-06-29,NaN,NaN,НРЗБ БЛОК,СООТ,CH-SPP-KKC/OVS-006,NaN,NaN,NaN
2,СП,СП:ЦВКС,КТГ,Комплекс конвертера N 1,CH-SPP-KKC/OVS-006,Комплекс конвертера N 1,17125032,Технологические плановые,Межплавочный период,NaN,межплавочный период,18.48,0.31,Ч,2020-06-28,...,12:19:36,X,NaN,M1,PMN_INT_USER,12:28:45,2020-06-28,NaN,NaN,НРЗБ БЛОК,СООТ,CH-SPP-KKC/OVS-006,NaN,NaN,NaN
3,СП,СП:ЦВКС,КТГ,Комплекс конвертера N 1,CH-SPP-KKC/OVS-006,Комплекс конвертера N 1,17124738,Технологические плановые,Межплавочный период,NaN,межплавочный период,20.18,0.34,Ч,2020-06-28,...,11:24:01,X,NaN,M1,PMN_INT_USER,11:30:44,2020-06-28,NaN,NaN,НРЗБ БЛОК,СООТ,CH-SPP-KKC/OVS-006,NaN,NaN,NaN
4,СП,СП:ЦВКС,КТГ,Комплекс конвертера N 1,CH-SPP-KKC/OVS-006,Комплекс конвертера N 1,17122692,Технологические плановые,Межплавочный период,NaN,межплавочный период,22.28,0.37,Ч,2020-06-28,...,06:13:19,X,NaN,M1,PMN_INT_USER,06:18:45,2020-06-28,NaN,NaN,НРЗБ БЛОК,СООТ,CH-SPP-KKC/OVS-006,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,СП,СП:ЦВКС,КТГ,Комплекс конвертера N 1,CH-SPP-KKC/OVS-006,Комплекс конвертера N 1,15503752,Технологические плановые,Межплавочный период,NaN,межплавочный период,18.08,0.30,Ч,2020-01-03,...,12:53:27,X,NaN,M1,PMN_INT_USER,13:01:58,2020-01-03,NaN,NaN,НРЗБ БЛОК,СООТ,CH-SPP-KKC/OVS-006,NaN,NaN,NaN
480,СП,СП:ЦВКС,КТГ,Комплекс конвертера N 1,CH-SPP-KKC/OVS-006,Комплекс конвертера N 1,15498549,Технологические плановые,Межплавочный период,NaN,межплавочный период,21.30,0.35,Ч,2020-01-02,...,14:45:07,X,NaN,M1,PMN_INT_USER,14:50:58,2020-01-02,NaN,NaN,НРЗБ БЛОК,СООТ,CH-SPP-KKC/OVS-006,NaN,NaN,NaN
481,СП,СП:ЦВКС,КТГ,Комплекс конвертера N 1,CH-SPP-KKC/OVS-006,Комплекс конвертера N 1,15495091,Технологические неплановые,Прочее,NaN,превышение времени на обработку плавки,0.62,0.01,Ч,2020-01-02,...,00:12:18,X,NaN,M1,PMN_INT_USER,00:14:57,2020-01-02,NaN,NaN,РПРВ БЛОК,СООТ,CH-SPP-KKC/OVS-006,"ПАО ""СЕВЕРСТАЛЬ""",ТЕХНОЛОГИ,СП: ЦВКС
482,СП,СП:ЦВКС,КТГ,Комплекс конвертера N 1,CH-SPP-KKC/OVS-006,Комплекс конвертера N 1,15492580,Технологические плановые,Межплавочный период,NaN,межплавочный период,25.35,0.42,Ч,2020-01-01,...,14:19:40,X,NaN,M1,PMN_INT_USER,14:20:57,2020-01-01,NaN,NaN,НРЗБ БЛОК,СООТ,CH-SPP-KKC/OVS-006,NaN,NaN,NaN


In [60]:
import datetime

start_dts = pd.Series([
    datetime.datetime.strptime(elem, '%Y-%m-%d%H:%M:%S')
    for elem in stops_data["НачалоНеисправн"].astype(str) +
    stops_data["ВремяНачалаНеис"]
], name = "start_datetime", dtype='datetime64[ns]')

end_dts = pd.Series([
    datetime.datetime.strptime(elem, '%Y-%m-%d%H:%M:%S')
    for elem in stops_data["ОкончНеиспр"].astype(str) +
    stops_data["ВремяУстрНеиспр"]
], name = "end_datetime", dtype='datetime64[ns]')

dscr = stops_data["ТекстГруппКодов"].astype(str) + " : " + stops_data["Текст кода код."].astype(str)
dscr.name = "description"

In [61]:
StopLogs = pd.concat([start_dts, end_dts, dscr], axis=1)[::-1]
StopLogs.index = range(StopLogs.shape[0])

In [83]:
for i, month in enumerate(
    ["Январь", "Февраль", "Март", "Апрель", "Май", "Июнь"]):
    print(month, '\n')
    month_stops = StopLogs[(StopLogs.start_datetime >= f"2020-0{i+1}")
                           & (StopLogs.start_datetime < f"2020-0{i+2}")]
    print(month_stops.description.value_counts(),'\n')
    print(f"Всего остановок : {month_stops.description.value_counts().values.sum()}")
    print("="*70,'\n')

Январь 

Технологические плановые : Межплавочный период                     49
Технологические плановые : Замена сталевыпускного блока (летки)     5
Технологические неплановые : Прочее                                 4
Технологические плановые : Прочее                                   1
Технологические плановые : Балансировка дымососа                    1
Технологические плановые : Уход за футеровкой                       1
Технические неплановые : Отказ                                      1
Технические плановые : ТР                                           1
Отсутствие загрузки агрегата : Вывод в резерв                       1
Технические плановые : ТО                                           1
Name: description, dtype: int64 

Всего остановок : 65

Февраль 

Технологические плановые : Межплавочный период                     65
Технологические неплановые : Прочее                                 7
Технологические плановые : Замена сталевыпускного блока (летки)     4
Технологические

In [65]:
StopLogs.to_csv("Jan_to_Jun_StopLogs.csv", index=0)